Interpolate only TLT file with angles

In [4]:
import pandas as pd  # Importing pandas for handling dataframes
import numpy as np   # Importing numpy for numerical operations

# Read the original .tlt file into a DataFrame
# `header=None` indicates that the file has no column headers
star = pd.read_csv('/path/to/original/file.tlt', header=None)

# Define the number of values to interpolate between existing rows
n = 1  # Number of interpolated rows between existing rows

# Calculate the new index size to accommodate both original and interpolated rows
# For each original row, (n+1) rows will be created, minus `n` for overlaps
new_index = pd.RangeIndex(len(star) * (n + 1) - n)

# Create a new DataFrame with the expanded index and same columns as `star`
# Initialize all entries in the new DataFrame with NaN
new_star = pd.DataFrame(np.nan, index=new_index, columns=star.columns)

# Calculate the positions in the new DataFrame where the original rows will be placed
# These positions are spaced by (n+1) intervals
ids = np.arange(len(star)) * (n + 1)

# Place the original data values into the corresponding positions in the new DataFrame
new_star.loc[ids] = star.values

# Perform linear interpolation to fill the NaN values
new_star = new_star.interpolate()

# Save the new DataFrame to a .tlt file with floating-point format
np.savetxt(f'/path/to/interpolated/file.tlt', new_star, fmt='%f')


Interpolate values in the wedge list, it saves also interpolated TLT file with angles

In [ ]:
import numpy as np             # Import numpy for numerical operations
import starfile                # Import starfile to read/write .star files
import pandas as pd            # Import pandas for handling dataframes
from cryocat import starfileio # Import starfileio for writing .star files using cryocat's functionality

# Define the TOMO_ID for selecting a specific TS
TOMO_ID = '1'  # Process only the data corresponding to this TS

# Define the number of interpolated tilts between existing tilts
iframe = 1  # Controls the number of interpolation steps (1 = 2^1 - 1 = 1 interpolated tilt)

# Read the original .star file containing the wedge list
star = starfile.read('/path/to/original/wedge_list.star')

# Filter the DataFrame to include only rows corresponding to the given TOMO_ID
star = star.drop(star[star["tomo_num"] != int(TOMO_ID)].index)

# Calculate the number of interpolated rows for each existing row
n = 2**iframe - 1  # The number of interpolated rows between original rows

# Create a new index for the expanded DataFrame to include interpolated rows
new_index = pd.RangeIndex(len(star) * (n + 1) - n)

# Create a new DataFrame with the expanded index and the same columns as the original
# Initialize all values as NaN (to later fill with interpolated values)
new_star = pd.DataFrame(np.nan, index=new_index, columns=star.columns)

# Calculate the positions in the new DataFrame where original rows will be placed
# Original data is spaced by `(n + 1)` intervals
ids = np.arange(len(star)) * (n + 1)

# Place the original data into the corresponding positions in the new DataFrame
new_star.loc[ids] = star.values

# Perform linear interpolation to fill the NaN values in the DataFrame
new_star = new_star.interpolate()

# Save the interpolated tilt angles to a .tlt file in floating-point format
np.savetxt(f'/path/to/interpolated/file.tlt', new_star['tilt_angle'], fmt='%f')

# Write the interpolated data to a new .star file
# Specify the data block name as "data_stopgap_wedgelist" and disable column numbering
starfileio.Starfile.write(
    [new_star], 
    f'/path/to/interpolated/wedge_list.star', 
    specifiers=["data_stopgap_wedgelist"], 
    number_columns=False
)
